In [22]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids, and separate the dataset:

In [23]:
from implementations import *

In [24]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [25]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [42]:
logistic = False
tX_list, ids_list, y_list = separate_dataset(tX, ids, y, logistic)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test) 

In [43]:
tX_list, tX_test_list, y_list = clean_data(tX_list, tX_test_list, y_list)

## GRID SEARCH RIDGE REGRESSION

In [55]:
function = ridge_regression
degree_vec = []
lambda_vec = []
for i in range(6):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, _ = grid_search(y_list[i], tX_list[i], function, 4, degrees= range(1, 10), lambdas=np.logspace(-8,0,15), gammas = np.arange(1), dataset = i)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Loss: {lo}'.format(lo = rmse_te))

Dataset part 0:
(array([8]), array([0]), array([3]))
     Best degree: 9
     Best lambda: 5.179474679231213e-07
     Loss: 6198.0
Dataset part 1:
(array([8]), array([0]), array([7]))
     Best degree: 9
     Best lambda: 0.0001
     Loss: 14892.75
Dataset part 2:
(array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
     Best degree: 7
     Best lambda: 1e-08
     Loss: 1734.75
Dataset part 3:
(array([8]), array([0]), array([9]))
     Best degree: 9
     Best lambda: 0.001389495494373136
     Loss: 13812.0
Dataset part 4:
(array([4, 4, 4, 4, 4, 4]), array([0, 0, 0, 0, 0, 0]), array([1, 2, 3, 4, 5, 6]))
     Best degree: 5
     Best lambda: 3.7275937203149377e-08
     Loss: 1007.0
Dataset part 5:
(array([8]), array([0]), array([7]))
     Best degree: 9
     Best lambda: 0.0001
     Loss: 13948.0


In [56]:
weights_list = []
loss_list = []
mat_tX_test_list = []

for i in range(6):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], tX_test_list[i], i)
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)

In [57]:
# Eval

y_pred_list = separated_eval(weights_list, mat_tX_test_list, logistic) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3], y_pred_list[4], y_pred_list[5]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3], ids_test_list[4], ids_test_list[5]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)